In [1]:
!pip install ultralytics torchvision  # Install YOLOv8 and Faster R-CNN dependencies

from ultralytics import YOLO
import os
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 17.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:


from ultralytics import YOLO
import os
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define transformations
train_transforms = T.Compose([
    T.Resize((128, 128)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transforms = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define RoadSignDataset
class RoadSignDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transforms=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transforms = transforms
        self.image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        label_file_name = os.path.splitext(self.image_files[idx])[0] + '.txt'
        label_path = os.path.join(self.labels_dir, label_file_name)

        # Load bounding boxes
        boxes, labels = [], []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    class_id = int(data[0])
                    x_center, y_center, width, height = map(float, data[1:])
                    xmin, ymin = x_center - width / 2, y_center - height / 2
                    xmax, ymax = x_center + width / 2, y_center + height / 2
                    boxes.append([xmin, ymin, xmax, ymax])
                    labels.append(class_id)

        # If no boxes, return empty tensors
        if not boxes:
            boxes, labels = torch.zeros((0, 4), dtype=torch.float32), torch.zeros((0,), dtype=torch.int64)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            image = self.transforms(image)

        return image, target

# Initialize datasets and loaders
train_dataset = RoadSignDataset(
    images_dir='/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/train/images',
    labels_dir='/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/train/labels',
    transforms=train_transforms
)
valid_dataset = RoadSignDataset(
    images_dir='/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/valid/images',
    labels_dir='/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/valid/labels',
    transforms=valid_transforms
)

train_loader = DataLoader(train_dataset, batch_size=48, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=48, shuffle=False)




In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torch.utils.data import DataLoader
from ultralytics import YOLO
import os
from tqdm import tqdm

In [ ]:
# Step 1: Load and Fine-tune Faster R-CNN with pretrained weights
# Load Faster R-CNN model with pretrained backbone on COCO dataset
weights = FasterRCNN_ResNet50_FPN_Weights.COCO_V1
faster_rcnn_model = fasterrcnn_resnet50_fpn(weights=weights)  # Load pre-trained weights
faster_rcnn_model.to(device)

# Adjust the classifier head to match the number of classes in your dataset
num_classes = 25  # Change to match your dataset
in_features = faster_rcnn_model.roi_heads.box_predictor.cls_score.in_features
faster_rcnn_model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Define optimizer and hyperparameters for Faster R-CNN
faster_rcnn_optimizer = torch.optim.SGD(faster_rcnn_model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 50  # Set number of epochs as needed


In [ ]:
# Custom collate function to handle varying sizes of targets
def collate_fn(batch):
    return tuple(zip(*batch))

# Define DataLoaders for training and validation, with custom collate function
# Make sure to replace `train_dataset` and `valid_dataset` with your datasets
train_loader = DataLoader(train_dataset, batch_size=48, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=48, shuffle=False, collate_fn=collate_fn)


In [ ]:
# Define a generic training function
def train_model(model, optimizer, loader, device, num_epochs=50, model_name="Model"):
    model.to(device)  # Ensure model is on the correct device
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, targets in tqdm(loader, desc=f"{model_name} - Training Epoch {epoch + 1}"):
            # Move images and targets to device
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images, targets)
            loss = sum(loss.to(device) for loss in outputs.values())  # Ensure loss is on the same device
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(loader)
        print(f"{model_name} - Epoch [{epoch + 1}/{num_epochs}] Loss: {avg_epoch_loss:.4f}")

        # Save model checkpoint after each epoch
        torch.save(model.state_dict(), f"/content/drive/MyDrive/DL LAB/{model_name}_epoch_{epoch + 1}.pth")
        print(f"{model_name} checkpoint saved after epoch {epoch + 1}")


In [ ]:
# Train Faster R-CNN
train_model(faster_rcnn_model, faster_rcnn_optimizer, train_loader, device, num_epochs=num_epochs, model_name="Faster_RCNN")


Faster_RCNN - Training Epoch 1: 100%|██████████| 111/111 [1:05:10<00:00, 35.23s/it]


Faster_RCNN - Epoch [1/50] Loss: nan
Faster_RCNN checkpoint saved after epoch 1


Faster_RCNN - Training Epoch 2: 100%|██████████| 111/111 [02:30<00:00,  1.35s/it]


Faster_RCNN - Epoch [2/50] Loss: nan
Faster_RCNN checkpoint saved after epoch 2


Faster_RCNN - Training Epoch 3: 100%|██████████| 111/111 [02:30<00:00,  1.36s/it]


...

Faster_RCNN - Epoch [48/50] Loss: nan
Faster_RCNN checkpoint saved after epoch 48


Faster_RCNN - Training Epoch 49: 100%|██████████| 111/111 [02:29<00:00,  1.35s/it]


Faster_RCNN - Epoch [49/50] Loss: nan
Faster_RCNN checkpoint saved after epoch 49


Faster_RCNN - Training Epoch 50: 100%|██████████| 111/111 [02:29<00:00,  1.35s/it]


Faster_RCNN - Epoch [50/50] Loss: nan
Faster_RCNN checkpoint saved after epoch 50


In [ ]:
# Step 2: Generate Predictions with Faster R-CNN for Training and Validation Sets
# Function to get Faster R-CNN predictions and save them in YOLO format
def generate_predictions(model, data_loader, output_dir):
    model.eval()
    os.makedirs(output_dir, exist_ok=True)

    with torch.no_grad():
        for i, (images, _) in enumerate(data_loader):  # Assuming dataset provides images only
            images = [image.to(device) for image in images]
            outputs = model(images)

            for j, output in enumerate(outputs):
                img_id = i * len(images) + j
                img_boxes = output['boxes'].cpu().numpy()
                img_labels = output['labels'].cpu().numpy()

                # Save in YOLO format
                with open(os.path.join(output_dir, f"{img_id}.txt"), 'w') as f:
                    for label, box in zip(img_labels, img_boxes):
                        x_center = (box[0] + box[2]) / 2
                        y_center = (box[1] + box[3]) / 2
                        width = box[2] - box[0]
                        height = box[3] - box[1]
                        f.write(f"{label} {x_center} {y_center} {width} {height}\n")

# Generate predictions for both training and validation sets
generate_predictions(faster_rcnn_model, train_loader, "/content/drive/MyDrive/DL LAB/yolo_format_train_labels")
generate_predictions(faster_rcnn_model, valid_loader, "/content/drive/MyDrive/DL LAB/yolo_format_valid_labels")


NameError: name 'faster_rcnn_model' is not defined

In [ ]:
# Step 4: Fine-tune YOLOv11 with YOLOv8 Weights
yolo_v11_model = YOLO('/content/drive/MyDrive/DL LAB/best_v8.pt').to(device)  # Initialize YOLOv11 with fine-tuned YOLOv8 weights

yolo_v11_model.train(
    data="/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml",  # Path to updated data.yaml
    epochs=50,  # Additional fine-tuning epochs for YOLOv11
    imgsz=128,
    batch=25,
)

# Save the fine-tuned YOLOv11 model weights
saved_yolo_v11_path = "/content/drive/MyDrive/DL LAB/best_v11.pt"
print(f"YOLOv11 model fine-tuned and weights saved as {saved_yolo_v11_path}.")


NameError: name 'YOLO' is not defined

In [ ]:
# Step 3: Train YOLOv8 with the enriched dataset
# Initialize YOLOv8 model with pretrained weights
yolo_v8_model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is the pre-trained YOLOv8 model
yolo_v8_model.to(device)

# Define the YOLOv8 data configuration to use both original and new labels
yolo_v8_model.train(
    data="/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml",  # Path to data.yaml with updated paths
    epochs=50,  # Set a reasonable number of epochs for training
    imgsz=128,  # Image size to match previous setup
    batch=25,   # Batch size
)

# Save the fine-tuned YOLOv8 model weights
saved_yolo_v8_path = "/content/drive/MyDrive/DL LAB/best_v8.pt"
print(f"YOLOv8 model fine-tuned and weights saved as {saved_yolo_v8_path}.")


100%|██████████| 6.25M/6.25M [00:00<00:00, 175MB/s]

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml, epochs=50, time=None, patience=100, batch=25, imgsz=128, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, 

100%|██████████| 755k/755k [00:00<00:00, 115MB/s]


Overriding model.yaml nc=80 with nc=25

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 370MB/s]


...

      50/50        11G      0.505     0.3724     0.9058         28        128: 100%|██████████| 213/213 [00:20<00:00, 10.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  5.95it/s]

                   all        623        801      0.847      0.808      0.857      0.701



50 epochs completed in 0.324 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,010,523 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  4.15it/s]


                   all        623        801      0.875      0.791      0.858      0.702
-Road narrows on right         15         15      0.948      0.933      0.988      0.888
     Attention Please-         23         23      0.957      0.978      0.993      0.889
    Beware of children         23         23      0.964      0.826      0.837      0.766
CYCLE ROUTE AHEAD WARNING         29         29      0.968      0.931      0.989      0.883
             Crosswalk         42         43      0.871      0.783      0.847      0.601
Dangerous Left Curve Ahead         24         24      0.613      0.708      0.762      0.609
Dangerous Rright Curve Ahead         30         30      0.869      0.882      0.937      0.785
              No Entry         26         27      0.946      0.963      0.975      0.845
        No_Over_Taking         23         23      0.966      0.783      0.873      0.665
          One way road         27         27      0.919       0.84      0.907      0.774
      Sp

In [ ]:
from ultralytics import YOLO  # Assuming you are using the ultralytics library for YOLO
# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Paths
faster_rcnn_weights_dir = "/content/drive/MyDrive/DL LAB"  # Path to saved weights directory
yolo_data_config = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml"  # Path to data.yaml

# Paths to Faster R-CNN YOLO-formatted label directories
train_labels_dir = "/content/drive/MyDrive/DL LAB/yolo_format_train_labels"
valid_labels_dir = "/content/drive/MyDrive/DL LAB/yolo_format_valid_labels"

# Step 1: Train YOLOv8 with the enriched dataset (Using Faster R-CNN generated labels)
# Initialize YOLOv8 model with pre-trained weights and move to the device
yolo_v8_model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is the pre-trained YOLOv8 model
yolo_v8_model.to(device)

# Update the data.yaml file to point to the new label directories (train and valid) generated by Faster R-CNN
# You may need to manually update data.yaml if not done already

# Define the YOLOv8 data configuration to use both original and new labels
yolo_v8_model.train(
    data=yolo_data_config,  # Path to data.yaml with updated paths to train_labels_dir and valid_labels_dir
    epochs=50,  # Set a reasonable number of epochs for training
    imgsz=128,  # Image size to match previous setup
    batch=25,   # Batch size
)

# Save the fine-tuned YOLOv8 model weights
saved_yolo_v8_path = f"{faster_rcnn_weights_dir}/best_v8.pt"
yolo_v8_model.save(saved_yolo_v8_path)
print(f"YOLOv8 model fine-tuned and weights saved as {saved_yolo_v8_path}.")


# Step 2: Fine-tune YOLOv11 with YOLOv8 Weights (Using the previously trained YOLOv8 weights)
# Initialize YOLOv11 with fine-tuned YOLOv8 weights and move to the device
yolo_v11_model = YOLO(saved_yolo_v8_path).to(device)  # Using fine-tuned YOLOv8 weights as a starting point

yolo_v11_model.train(
    data=yolo_data_config,  # Path to updated data.yaml with YOLOv8 enriched labels
    epochs=50,  # Additional fine-tuning epochs for YOLOv11
    imgsz=128,  # Match image size
    batch=25,   # Batch size
)

# Save the fine-tuned YOLOv11 model weights
saved_yolo_v11_path = f"{faster_rcnn_weights_dir}/best_v11.pt"
yolo_v11_model.save(saved_yolo_v11_path)
print(f"YOLOv11 model fine-tuned and weights saved as {saved_yolo_v11_path}.")


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml, epochs=50, time=None, patience=100, batch=25, imgsz=128, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript,

100%|██████████| 755k/755k [00:00<00:00, 14.3MB/s]


Overriding model.yaml nc=80 with nc=25

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.6MB/s]


AMP: checks passed ✅


...

      50/50     0.329G     0.4779      0.307     0.8971         28        128: 100%|██████████| 213/213 [00:20<00:00, 10.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  5.67it/s]

                   all        623        801      0.884      0.823      0.874      0.718



50 epochs completed in 0.334 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,010,523 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  4.03it/s]


                   all        623        801      0.866      0.834      0.873      0.718
-Road narrows on right         15         15      0.888          1      0.987      0.888
     Attention Please-         23         23      0.932          1      0.995      0.883
    Beware of children         23         23      0.955      0.826      0.857      0.778
CYCLE ROUTE AHEAD WARNING         29         29      0.977      0.931      0.979       0.87
             Crosswalk         42         43      0.836      0.829       0.83      0.592
Dangerous Left Curve Ahead         24         24      0.657       0.75      0.724      0.611
Dangerous Rright Curve Ahead         30         30      0.826      0.867      0.918      0.783
              No Entry         26         27      0.935      0.963      0.985      0.834
        No_Over_Taking         23         23       0.94       0.87       0.89      0.686
          One way road         27         27      0.916       0.81      0.904      0.785
      Sp

In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from ultralytics import YOLO

# Set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define transformations to resize and normalize image
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match YOLOv11 input size
    transforms.ToTensor()  # Convert to tensor (automatically scales to 0-1 range)
])

# Define class names based on your data.yaml
class_names = [
    '-Road narrows on right', 'Attention Please-', 'Beware of children', 'CYCLE ROUTE AHEAD WARNING', 'Crosswalk',
    'Dangerous Left Curve Ahead', 'Dangerous Right Curve Ahead', 'No Entry', 'No_Over_Taking', 'One way road',
    'Speed bump ahead', 'Speed limit', 'Speed limit 100 Kph', 'Speed limit 120 Kph', 'Speed limit 20 Kph',
    'Speed limit 30 Kph', 'Speed limit 40 Kph', 'Speed limit 50 Kph', 'Speed limit 60 Kph', 'Speed limit 70 Kph',
    'Speed limit 80 Kph', 'Speed limit 90 Kph', 'Stop_Sign', 'Uneven Road', 'roundabout'
]
class_mapping = {i: name for i, name in enumerate(class_names)}

# Load YOLOv11 model with the fine-tuned weights
yolo_v11_model_path = "/content/drive/MyDrive/DL LAB/best_v11.pt"  # Path to the fine-tuned YOLOv11 weights
yolo_v11_model = YOLO(yolo_v11_model_path).to(device)

# Directory for test images (unlabeled)
test_images_dir = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/test"  # Update this to match your test images directory
test_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg') or f.endswith('.png')]

# List to store predictions
predictions = []

# Run inference and collect predictions
for img_name in test_images:
    img_path = os.path.join(test_images_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    image = test_transform(image).unsqueeze(0).to(device)  # Apply transformations and add batch dimension

    # YOLOv11 inference
    with torch.no_grad():
        yolo_results = yolo_v11_model(image)

        # Process output to check for detections
        if len(yolo_results[0].boxes) > 0:
            # Get the class ID for the first detection
            pred_class = int(yolo_results[0].boxes[0].cls)
            pred_class_name = class_mapping.get(pred_class, 'none')
        else:
            # If no detections, set class as 'none'
            pred_class_name = 'none'

        # Append filename and predicted class to predictions list
        predictions.append({"filename": img_name, "class": pred_class_name})

# Save predictions to CSV
submission_path = "/content/drive/MyDrive/DL LAB/submission.csv"  # Path to save the submission file
submission_df = pd.DataFrame(predictions)
submission_df.to_csv(submission_path, index=False)
print(f"Submission CSV with class labels created successfully at {submission_path}.")



0: 128x128 1 Speed limit 30 Kph, 12.0ms
Speed: 0.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 90 Kph, 7.4ms
Speed: 0.0ms preprocess, 7.4ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 7.2ms
Speed: 0.0ms preprocess, 7.2ms inference, 1.4ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 9.6ms
Speed: 0.0ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 8.1ms
Speed: 0.0ms preprocess, 8.1ms inference, 1.7ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Crosswalk, 1 Speed limit 20 Kph, 7.6ms
Speed: 0.0ms preprocess, 7.6ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 8.2ms
Speed: 0.0ms preprocess, 8.2ms inference, 1.6ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Stop_Sign, 8.5ms
Sp

# Hyper Parameter tunning


In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Paths to saved weights
faster_rcnn_saved_weights = "/content/drive/MyDrive/DL LAB/Faster_RCNN_epoch_50.pth"

# Load Faster R-CNN Model with Modified `box_predictor` Layer for 25 Classes
faster_rcnn_model = fasterrcnn_resnet50_fpn(pretrained=False)
num_classes = 25  # Change this to match the number of classes in your dataset

# Modify the `box_predictor` to match the class count
in_features = faster_rcnn_model.roi_heads.box_predictor.cls_score.in_features
faster_rcnn_model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Now, load the saved weights into the modified model
faster_rcnn_model.load_state_dict(torch.load(faster_rcnn_saved_weights), strict=False)
faster_rcnn_model.to(device)

print("Model loaded successfully with adjusted class count.")


Model loaded successfully with adjusted class count.


In [ ]:
import torch
from ultralytics import YOLO
import torchvision
from torch.optim.lr_scheduler import CosineAnnealingLR

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Paths to saved weights
#faster_rcnn_saved_weights = "/content/drive/MyDrive/DL LAB/Faster_RCNN_epoch_50.pth"
yolo_v8_saved_weights = "/content/drive/MyDrive/DL LAB/best_v8.pt"
yolo_v11_saved_weights = "/content/drive/MyDrive/DL LAB/best_v11.pt"

# Data config path
yolo_data_config = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml"

# # Load Faster R-CNN Model with Saved Weights
# faster_rcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
# faster_rcnn_model.load_state_dict(torch.load(faster_rcnn_saved_weights))
# faster_rcnn_model.to(device)

# Load YOLOv8 and YOLOv11 Models with Saved Weights
yolo_v8_model = YOLO(yolo_v8_saved_weights).to(device)
yolo_v11_model = YOLO(yolo_v11_saved_weights).to(device)

# Set Up Hyperparameters for Fine-Tuning
# Define a new learning rate and use a scheduler
faster_rcnn_optimizer = torch.optim.SGD(faster_rcnn_model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = CosineAnnealingLR(faster_rcnn_optimizer, T_max=10)  # Adjust T_max based on epochs

# Updated batch size and number of epochs
new_batch_size = 32
new_num_epochs = 80


In [ ]:
from tqdm import tqdm
new_batch_size = 45
new_num_epochs = 60
# Custom collate function to handle varying sizes of targets
def collate_fn(batch):
    return tuple(zip(*batch))

# Define DataLoader with the updated batch size
train_loader = DataLoader(train_dataset, batch_size=new_batch_size, shuffle=True, collate_fn=collate_fn)

# Define a function to fine-tune the Faster R-CNN model
def train_faster_rcnn(model, optimizer, scheduler, loader, device, num_epochs=80, model_name="Faster_RCNN"):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, targets in tqdm(loader, desc=f"{model_name} - Training Epoch {epoch + 1}"):
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()
            outputs = model(images, targets)
            loss = sum(loss.to(device) for loss in outputs.values())
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate via scheduler

            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(loader)
        print(f"{model_name} - Epoch [{epoch + 1}/{num_epochs}] Loss: {avg_epoch_loss:.4f}")

        # Save model checkpoint after each epoch
        torch.save(model.state_dict(), f"/content/drive/MyDrive/DL LAB/{model_name}_epoch_{epoch + 1}.pth")
        print(f"{model_name} checkpoint saved after epoch {epoch + 1}")

# Fine-tune Faster R-CNN with updated parameters
train_faster_rcnn(faster_rcnn_model, faster_rcnn_optimizer, lr_scheduler, train_loader, device, num_epochs=new_num_epochs)


Faster_RCNN - Training Epoch 1:   0%|          | 0/119 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: roi_align_backward_kernel does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Faster_RCNN - Training Epoch 1: 100%|██████████| 119/119 [20:23<00:00, 10.28s/it]


Faster_RCNN - Epoch [1/60] Loss: nan
Faster_RCNN checkpoint saved after epoch 1


Faster_RCNN - Training Epoch 2: 100%|██████████| 119/119 [02:46<00:00,  1.40s/it]


Faster_RCNN - Epoch [2/60] Loss: nan
Faster_RCNN checkpoint saved after epoch 2


Faster_RCNN - Training Epoch 3: 100%|██████████| 119/119 [02:44<00:00,  1.38s/it]


...

Faster_RCNN - Epoch [58/60] Loss: nan
Faster_RCNN checkpoint saved after epoch 58


Faster_RCNN - Training Epoch 59: 100%|██████████| 119/119 [02:44<00:00,  1.38s/it]


Faster_RCNN - Epoch [59/60] Loss: nan
Faster_RCNN checkpoint saved after epoch 59


Faster_RCNN - Training Epoch 60: 100%|██████████| 119/119 [02:44<00:00,  1.38s/it]


Faster_RCNN - Epoch [60/60] Loss: nan
Faster_RCNN checkpoint saved after epoch 60


In [ ]:
# Custom collate function to handle varying sizes of targets
def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)


In [ ]:
import os

import os
import torch

# Function to generate predictions with Faster R-CNN and save them in YOLO format
def generate_predictions(model, data_loader, output_dir):
    # Ensure model is in evaluation mode and output directory exists
    model.eval()
    os.makedirs(output_dir, exist_ok=True)

    with torch.no_grad():
        for i, (images, _) in enumerate(data_loader):
            images = [image.to(device) for image in images]
            outputs = model(images)

            for j, output in enumerate(outputs):
                img_id = i * len(images) + j
                img_boxes = output['boxes'].cpu().numpy()
                img_labels = output['labels'].cpu().numpy()

                # Save in YOLO format
                with open(os.path.join(output_dir, f"{img_id}.txt"), 'w') as f:
                    for label, box in zip(img_labels, img_boxes):
                        # Calculate YOLO format: x_center, y_center, width, height
                        x_center = (box[0] + box[2]) / 2
                        y_center = (box[1] + box[3]) / 2
                        width = box[2] - box[0]
                        height = box[3] - box[1]
                        f.write(f"{label} {x_center} {y_center} {width} {height}\n")
                print(f"Saved predictions for image {img_id} in YOLO format.")


# Redefine DataLoader with updated collate_fn for both train and validation loaders
train_loader = DataLoader(train_dataset, batch_size=new_batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=new_batch_size, shuffle=False, collate_fn=collate_fn)

# Generate predictions for both training and validation sets in YOLO format
generate_predictions(faster_rcnn_model, train_loader, "/content/drive/MyDrive/DL LAB/yolo_format_train_labels")
generate_predictions(faster_rcnn_model, valid_loader, "/content/drive/MyDrive/DL LAB/yolo_format_valid_labels")


Streaming output truncated to the last 5000 lines.
Saved predictions for image 948 in YOLO format.
Saved predictions for image 949 in YOLO format.
Saved predictions for image 950 in YOLO format.
Saved predictions for image 951 in YOLO format.
Saved predictions for image 952 in YOLO format.
Saved predictions for image 953 in YOLO format.
Saved predictions for image 954 in YOLO format.
Saved predictions for image 955 in YOLO format.
Saved predictions for image 956 in YOLO format.
Saved predictions for image 957 in YOLO format.
Saved predictions for image 958 in YOLO format.
Saved predictions for image 959 in YOLO format.
Saved predictions for image 960 in YOLO format.
Saved predictions for image 961 in YOLO format.
Saved predictions for image 962 in YOLO format.
Saved predictions for image 963 in YOLO format.
Saved predictions for image 964 in YOLO format.
Saved predictions for image 965 in YOLO format.
Saved predictions for image 966 in YOLO format.
Saved predictions for image 967 in YO

In [ ]:
# YOLO data config path (assuming paths to generated labels are set in data.yaml)
yolo_data_config = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml"

# Fine-tune YOLOv8 with updated parameters
yolo_v8_model.train(
    data=yolo_data_config,  # Path to data.yaml with updated paths to train and valid labels
    epochs=80,  # Set to match the new number of epochs
    imgsz=128,              # Image size
    batch=45,   # Updated batch size
    lr0=0.001               # Initial learning rate
)

# Save the fine-tuned YOLOv8 model weights
saved_yolo_v8_path = "/content/drive/MyDrive/DL LAB/best_v8_finetuned.pt"
yolo_v8_model.save(saved_yolo_v8_path)
print(f"YOLOv8 model fine-tuned and weights saved as {saved_yolo_v8_path}.")

# Fine-tune YOLOv11 with YOLOv8 weights as starting point
yolo_v11_model = YOLO(saved_yolo_v8_path).to(device)  # Initialize with YOLOv8 weights
yolo_v11_model.train(
    data=yolo_data_config,  # Path to data.yaml
    epochs=80,  # Set to match the new number of epochs
    imgsz=128,              # Image size
    batch=45,   # Updated batch size
    lr0=0.001               # Initial learning rate
)

# Save the fine-tuned YOLOv11 model weights
saved_yolo_v11_path = "/content/drive/MyDrive/DL LAB/best_v11_finetuned.pt"
yolo_v11_model.save(saved_yolo_v11_path)
print(f"YOLOv11 model fine-tuned and weights saved as {saved_yolo_v11_path}.")


engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/DL LAB/best_v8.pt, data=/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml, epochs=80, time=None, patience=100, batch=45, imgsz=128, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

train: Scanning /content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/train/labels.cache... 5325 images, 12 backgrounds, 0 corrupt: 100%|██████████| 5325/5325 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/valid/labels.cache... 623 images, 1 backgrounds, 0 corrupt: 100%|██████████| 623/623 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000345, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0003515625), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 128 train, 128 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      80/80      2.54G     0.4362     0.2523     0.8852         22        128: 100%|██████████| 119/119 [00:17<00:00,  6.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.31it/s]

                   all        623        801      0.912      0.813      0.885      0.725



80 epochs completed in 0.465 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,010,523 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.35it/s]


                   all        623        801      0.911      0.813      0.885      0.726
-Road narrows on right         15         15      0.945      0.933      0.971      0.875
     Attention Please-         23         23       0.99          1      0.995      0.907
    Beware of children         23         23      0.977      0.826      0.874      0.779
CYCLE ROUTE AHEAD WARNING         29         29      0.978      0.931      0.983       0.88
             Crosswalk         42         43      0.876      0.814      0.869      0.652
Dangerous Left Curve Ahead         24         24       0.77      0.708      0.764      0.643
Dangerous Rright Curve Ahead         30         30      0.835      0.846      0.916      0.792
              No Entry         26         27      0.949      0.963      0.984      0.852
        No_Over_Taking         23         23          1      0.859      0.923      0.678
          One way road         27         27      0.885      0.778      0.906      0.767
      Sp

In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from ultralytics import YOLO

# Set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define transformations to resize and normalize image
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match YOLOv11 input size
    transforms.ToTensor()  # Convert to tensor (automatically scales to 0-1 range)
])

# Define class names based on your data.yaml
class_names = [
    '-Road narrows on right', 'Attention Please-', 'Beware of children', 'CYCLE ROUTE AHEAD WARNING', 'Crosswalk',
    'Dangerous Left Curve Ahead', 'Dangerous Right Curve Ahead', 'No Entry', 'No_Over_Taking', 'One way road',
    'Speed bump ahead', 'Speed limit', 'Speed limit 100 Kph', 'Speed limit 120 Kph', 'Speed limit 20 Kph',
    'Speed limit 30 Kph', 'Speed limit 40 Kph', 'Speed limit 50 Kph', 'Speed limit 60 Kph', 'Speed limit 70 Kph',
    'Speed limit 80 Kph', 'Speed limit 90 Kph', 'Stop_Sign', 'Uneven Road', 'roundabout'
]
class_mapping = {i: name for i, name in enumerate(class_names)}

# Load YOLOv11 model with the fine-tuned weights
yolo_v11_model_path = "/content/drive/MyDrive/DL LAB/best_v8_finetuned.pt"  # Path to the fine-tuned YOLOv11 weights
yolo_v11_model = YOLO(yolo_v11_model_path).to(device)

# Directory for test images (unlabeled)
test_images_dir = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/test"  # Update this to match your test images directory
test_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg') or f.endswith('.png')]

# List to store predictions
predictions = []

# Run inference and collect predictions
for img_name in test_images:
    img_path = os.path.join(test_images_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    image = test_transform(image).unsqueeze(0).to(device)  # Apply transformations and add batch dimension

    # YOLOv11 inference
    with torch.no_grad():
        yolo_results = yolo_v11_model(image)

        # Process output to check for detections
        if len(yolo_results[0].boxes) > 0:
            # Get the class ID for the first detection
            pred_class = int(yolo_results[0].boxes[0].cls)
            pred_class_name = class_mapping.get(pred_class, 'none')
        else:
            # If no detections, set class as 'none'
            pred_class_name = 'none'

        # Append filename and predicted class to predictions list
        predictions.append({"filename": img_name, "class": pred_class_name})

# Save predictions to CSV
submission_path = "/content/drive/MyDrive/DL LAB/submission.csv"  # Path to save the submission file
submission_df = pd.DataFrame(predictions)
submission_df.to_csv(submission_path, index=False)
print(f"Submission CSV with class labels created successfully at {submission_path}.")


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


TypeError: model='/content/drive/MyDrive/DL LAB/Faster_RCNN_epoch_60.pth' should be a *.pt PyTorch model to run this method, but is a different format. PyTorch models can train, val, predict and export, i.e. 'model.train(data=...)', but exported formats like ONNX, TensorRT etc. only support 'predict' and 'val' modes, i.e. 'yolo predict model=yolov8n.onnx'.
To run CUDA or MPS inference please pass the device argument directly in your inference command, i.e. 'model.predict(source=..., device=0)'

In [ ]:
# Define the path to your data configuration and the saved weights from the last best results
yolo_data_config = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml"
last_saved_yolo_v8_weights = "/content/drive/MyDrive/DL LAB/best_v8_finetuned.pt"
last_saved_yolo_v11_weights = "/content/drive/MyDrive/DL LAB/best_v11_finetuned.pt"

# Load YOLOv8 model with the last saved weights and resume training
yolo_v8_model = YOLO('/content/drive/MyDrive/DL LAB/best_v11_finetuned.pt').to(device)  # Load YOLOv8 with last saved weights
yolo_v8_model.train(
    data=yolo_data_config,  # Path to data.yaml with updated paths to train and valid labels
    epochs=100,  # Number of additional epochs to run from the last checkpoint
    imgsz=128,  # Image size
    batch=45,   # Batch size
    lr0=0.001   # Initial learning rate
)

# Save the updated YOLOv8 model weights
updated_yolo_v8_path = "/content/drive/MyDrive/DL LAB/best_v8_finetuned_continued.pt"
yolo_v8_model.save(updated_yolo_v8_path)
print(f"YOLOv8 model continued training and weights saved as {updated_yolo_v8_path}.")

# Load YOLOv11 model with the last saved YOLOv8 weights as the starting point
yolo_v11_model = YOLO(updated_yolo_v8_path).to(device)  # Initialize YOLOv11 with YOLOv8 last checkpoint
yolo_v11_model.train(
    data=yolo_data_config,  # Path to data.yaml
    epochs=100,  # Additional epochs to continue training from the last checkpoint
    imgsz=128,  # Image size
    batch=45,   # Batch size
    lr0=0.001   # Initial learning rate
)

# Save the updated YOLOv11 model weights
updated_yolo_v11_path = "/content/drive/MyDrive/DL LAB/best_v11_finetuned_continued.pt"
yolo_v11_model.save(updated_yolo_v11_path)
print(f"YOLOv11 model continued training and weights saved as {updated_yolo_v11_path}.")


engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/DL LAB/best_v11_finetuned.pt, data=/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/data.yaml, epochs=100, time=None, patience=100, batch=45, imgsz=128, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

train: Scanning /content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/train/labels.cache... 5325 images, 12 backgrounds, 0 corrupt: 100%|██████████| 5325/5325 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/valid/labels.cache... 623 images, 1 backgrounds, 0 corrupt: 100%|██████████| 623/623 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000345, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0003515625), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 128 train, 128 val
Using 8 dataloader workers
Logging results to runs/detect/train7
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.56G     0.4051     0.2259     0.8766         20        128: 100%|██████████| 119/119 [00:17<00:00,  6.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.44it/s]

                   all        623        801      0.876      0.834      0.884      0.724



100 epochs completed in 0.579 hours.
Optimizer stripped from runs/detect/train8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train8/weights/best.pt, 6.2MB

Validating runs/detect/train8/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,010,523 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.36it/s]


                   all        623        801      0.882       0.83      0.883      0.724
-Road narrows on right         15         15      0.914      0.933      0.988      0.876
     Attention Please-         23         23      0.914          1      0.995      0.905
    Beware of children         23         23      0.955      0.826      0.877      0.784
CYCLE ROUTE AHEAD WARNING         29         29      0.958      0.931      0.972      0.859
             Crosswalk         42         43      0.876      0.814      0.874      0.628
Dangerous Left Curve Ahead         24         24      0.657      0.708      0.787       0.66
Dangerous Rright Curve Ahead         30         30      0.813      0.833       0.88      0.754
              No Entry         26         27      0.964      0.993      0.991      0.858
        No_Over_Taking         23         23          1      0.891      0.943      0.706
          One way road         27         27      0.921      0.852      0.901      0.754
      Sp

In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from ultralytics import YOLO

# Set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define transformations to resize and normalize image
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match YOLOv11 input size
    transforms.ToTensor()  # Convert to tensor (automatically scales to 0-1 range)
])

# Define class names based on your data.yaml
class_names = [
    '-Road narrows on right', 'Attention Please-', 'Beware of children', 'CYCLE ROUTE AHEAD WARNING', 'Crosswalk',
    'Dangerous Left Curve Ahead', 'Dangerous Right Curve Ahead', 'No Entry', 'No_Over_Taking', 'One way road',
    'Speed bump ahead', 'Speed limit', 'Speed limit 100 Kph', 'Speed limit 120 Kph', 'Speed limit 20 Kph',
    'Speed limit 30 Kph', 'Speed limit 40 Kph', 'Speed limit 50 Kph', 'Speed limit 60 Kph', 'Speed limit 70 Kph',
    'Speed limit 80 Kph', 'Speed limit 90 Kph', 'Stop_Sign', 'Uneven Road', 'roundabout'
]
class_mapping = {i: name for i, name in enumerate(class_names)}

# Load YOLOv11 model with the fine-tuned weights
yolo_v11_model_path = "/content/drive/MyDrive/DL LAB/best_v8_finetuned_continued.pt"  # Path to the fine-tuned YOLOv11 weights
yolo_v11_model = YOLO(yolo_v11_model_path).to(device)

# Directory for test images (unlabeled)
test_images_dir = "/content/drive/MyDrive/DL LAB/Road_Sign_Detection  3/test"  # Update this to match your test images directory
test_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg') or f.endswith('.png')]

# List to store predictions
predictions = []

# Run inference and collect predictions
for img_name in test_images:
    img_path = os.path.join(test_images_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    image = test_transform(image).unsqueeze(0).to(device)  # Apply transformations and add batch dimension

    # YOLOv11 inference
    with torch.no_grad():
        yolo_results = yolo_v11_model(image)

        # Process output to check for detections
        if len(yolo_results[0].boxes) > 0:
            # Get the class ID for the first detection
            pred_class = int(yolo_results[0].boxes[0].cls)
            pred_class_name = class_mapping.get(pred_class, 'none')
        else:
            # If no detections, set class as 'none'
            pred_class_name = 'none'

        # Append filename and predicted class to predictions list
        predictions.append({"filename": img_name, "class": pred_class_name})

# Save predictions to CSV
submission_path = "/content/drive/MyDrive/DL LAB/submission.csv"  # Path to save the submission file
submission_df = pd.DataFrame(predictions)
submission_df.to_csv(submission_path, index=False)
print(f"Submission CSV with class labels created successfully at {submission_path}.")



0: 128x128 1 Speed limit 30 Kph, 8.6ms
Speed: 0.0ms preprocess, 8.6ms inference, 2.0ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 90 Kph, 7.3ms
Speed: 0.0ms preprocess, 7.3ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 8.7ms
Speed: 0.0ms preprocess, 8.7ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 9.7ms
Speed: 0.0ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 8.6ms
Speed: 0.0ms preprocess, 8.6ms inference, 1.7ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Crosswalk, 1 Speed limit 20 Kph, 8.0ms
Speed: 0.0ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Speed limit 30 Kph, 7.5ms
Speed: 0.0ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 128)

0: 128x128 1 Stop_Sign, 7.6ms
Spee